## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [3]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [9]:
import findspark
findspark.init()

In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [11]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [13]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [14]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [16]:
# count
rdd.count()

1000

In [17]:
# first
rdd.first()

1

In [19]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [20]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [22]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [25]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([9.09741662e-01, 1.07509074e+00, 7.90031105e-01, 1.21835675e+00,
       2.69334029e+00, 3.31519203e-01, 8.63036611e-03, 2.07873620e+00,
       8.84327969e+00, 9.38814695e+00, 3.52445325e-01, 8.29290105e+00,
       1.04128177e+01, 1.07449647e+01, 1.22268506e+01, 1.74185028e+00,
       4.54314298e-01, 1.44040414e+01, 4.24809453e+00, 1.39450322e+01,
       3.46756019e+00, 1.75812153e+01, 4.62454702e+00, 2.11907851e+01,
       4.39789466e+00, 3.81231212e-01, 4.03959288e+00, 9.35010406e+00,
       7.68469404e-02, 1.22027252e+01, 9.86089088e+00, 2.92821238e+01,
       2.87568661e+01, 2.87261858e+01, 3.12567619e+00, 3.39981914e+01,
       3.27979547e+01, 7.17657260e-02, 2.62244192e+01, 3.02375403e-01,
       1.24347289e+01, 9.51323979e+00, 7.25062043e+00, 4.22949558e+01,
       1.84084259e+01, 2.15971301e+01, 1.29338533e+01, 2.78320076e+01,
       4.86831985e+01, 2.70493983e+01, 3.30801913e+01, 1.69588100e+01,
       2.80201054e+01, 5.38624307e+01, 4.72115424e+01, 2.32265442e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [27]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(5)

[0.9097416621892258,
 1.0750907385474997,
 0.7900311053352941,
 1.2183567492258631,
 2.6933402936140407]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [28]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [29]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.8369623292140878,
 0.9890834794636998,
 0.7268286169084706,
 1.120888209287794,
 2.4778730701249176,
 0.3049976671949442,
 0.007939936819568826,
 1.9124373063583469,
 8.135817315354902,
 8.637095197625007]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [32]:
discounted = price_items.map(lambda x : x*.90)

In [33]:
discounted.take(10)

[0.8187674959703033,
 0.9675816646927498,
 0.7110279948017647,
 1.096521074303277,
 2.4240062642526365,
 0.2983672831254889,
 0.007767329497404285,
 1.8708625823070784,
 7.958951721542839,
 8.449332258546201]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [34]:
price_items.map(lambda x : x*0.90).map(sales_tax).top(15)

[806.3306740237836,
 803.8051602607724,
 780.018068219554,
 762.86815441443,
 754.3655437619519,
 748.4482593928146,
 728.5029025780455,
 727.8273707102985,
 712.0781659537221,
 708.5102132595046,
 703.0631754066147,
 700.487428615305,
 697.3210834990548,
 694.816697467388,
 694.3081698836852]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [35]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [49]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(3))

1000
[(0.9097416621892258, 0.753266096292679), (1.0750907385474997, 0.8901751315173299), (0.7900311053352941, 0.6541457552176235)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [50]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(3))

2000
[0.9097416621892258, 0.753266096292679, 1.0750907385474997]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [47]:
# use the filter function
selected_items = discounted.filter(lambda x : x>300)

# calculate total remaining in inventory 
selected_items.count()

selected_items.take(10)

[328.7509461952149,
 322.7201581860288,
 319.6991431034418,
 306.50081431347803,
 313.16238229252343,
 327.95040862502276,
 343.17470009697655,
 355.2988801020333,
 331.87305471246884,
 364.83392660835807]

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [40]:
selected_items.reduce(lambda x,y : x + y)

151232.60230539506

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [41]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(4, 328.7509461952149),
 (34, 322.7201581860288),
 (44, 319.6991431034418),
 (32, 306.50081431347803),
 (4, 313.16238229252343),
 (4, 327.95040862502276),
 (22, 343.17470009697655)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [54]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y : x+y)
total_spent.take(10)

[(30, 3798.9549992867433),
 (50, 3065.404276872998),
 (10, 6825.315573756229),
 (20, 3431.365136337263),
 (40, 2255.9559452508774),
 (1, 4055.8340375690705),
 (21, 4614.281244970507),
 (11, 3840.2516165579314),
 (41, 1964.4929229072893),
 (31, 2335.694308743668)]

In [46]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(10, 6825.315573756229),
 (26, 6191.127018000689),
 (16, 5751.730314775039),
 (35, 5307.480334509566),
 (44, 5192.264847340997),
 (32, 5155.3425978628675),
 (21, 4614.281244970507),
 (37, 4608.911862438636),
 (43, 4319.589326248057),
 (4, 4184.787931804385),
 (33, 4103.71831092379),
 (1, 4055.8340375690705),
 (11, 3840.2516165579314),
 (30, 3798.9549992867433),
 (48, 3780.842510718174),
 (23, 3723.350159291574),
 (12, 3684.3404817827673),
 (20, 3431.365136337263),
 (6, 3272.5784109579668),
 (42, 3166.5831985497216),
 (27, 3109.39441895029),
 (50, 3065.404276872998),
 (38, 3055.428749350682),
 (17, 2864.698681393239),
 (9, 2852.9046898244087),
 (22, 2679.7833477211007),
 (28, 2424.3569215769576),
 (31, 2335.694308743668),
 (49, 2302.8420072396125),
 (19, 2259.7787726713614),
 (40, 2255.9559452508774),
 (29, 2174.7073496360954),
 (24, 2149.390261112555),
 (45, 2071.3358196111412),
 (5, 2056.8040219137483),
 (46, 2045.8775989444089),
 (18, 1972.0041075378253),
 (41, 1964.4929229072893),


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [48]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda x:x[1], reverse=True)

[(26, 12),
 (10, 12),
 (44, 11),
 (32, 11),
 (35, 11),
 (16, 11),
 (4, 10),
 (37, 10),
 (30, 9),
 (33, 9),
 (21, 9),
 (43, 9),
 (48, 8),
 (1, 8),
 (23, 8),
 (50, 8),
 (11, 8),
 (12, 8),
 (42, 7),
 (22, 6),
 (28, 6),
 (27, 6),
 (6, 6),
 (38, 6),
 (20, 6),
 (9, 6),
 (45, 5),
 (49, 5),
 (17, 5),
 (34, 4),
 (47, 4),
 (7, 4),
 (24, 4),
 (25, 4),
 (46, 4),
 (29, 4),
 (18, 4),
 (39, 4),
 (5, 4),
 (8, 4),
 (19, 4),
 (41, 4),
 (36, 4),
 (31, 4),
 (15, 3),
 (13, 3),
 (2, 3),
 (40, 3),
 (14, 2),
 (3, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
